<a href="https://colab.research.google.com/github/msbeigi/sale-analysis/blob/main/backery_sale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt


## Initiation and Data Understanding


## bakery_sales_2021-2022.csv

| Column        | Description               | Type     |
| ------------- | ------------------------- | -------- |
| -             | row index                 | Integer  |
| date          | date of sale              | Datetime |
| time          | time of sale              | Datetime |
| ticket_number | receipt number            | Integer  |
| Article       | product name              | String   |
| Quantity      | quantity of the product   | Integer  |
| UNIT_PRICE    | unit price of the product | Float    |


## weather_2021.01.01-2022.10.31.csv

| Column | Description                               | Type     |
| ------ | ----------------------------------------- | -------- |
| date   | The date of the measurement               | Datetime |
| tavg   | The average air temperature in °C         | Float    |
| tmin   | The minimum air temperature in °C         | Float    |
| tmax   | The maximum air temperature in °C         | Float    |
| prcp   | The daily precipitation total in mm       | Float    |
| snow   | The snow depth in mm                      | Float    |
| wdir   | The average wind direction in degrees (°) | Float    |
| wspd   | The average wind speed in km/h            | Float    |
| wpgt   | The peak wind gust in km/h                | Float    |
| pres   | The average sea-level air pressure in hPa | Float    |
| tsun   | The daily sunshine total in minutes (m)   | Float    |


In [5]:
weather=pd.read_csv("/content/drive/MyDrive/ML/Hand-on/Backery/weather_2021.01.01-2022.10.31.csv")
backery=pd.read_csv("/content/drive/MyDrive/ML/Hand-on/Backery/bakery_sales_2021-2022.csv")
display(backery.head(7))
display(weather.head(7))

,Unnamed: 0,date,time,ticket_number,Article,Quantity,UNIT_PRICE
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"
5,11,2021-01-02,09:25,150043.0,BAGUETTE,2.0,"0,90 €"
6,12,2021-01-02,09:25,150043.0,CROISSANT,3.0,"1,10 €"


,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2021-01-01,2.7,1.5,5.7,15.2,NaN,335.0,15.5,31.0,1008.6,NaN
1,2021-01-02,1.3,0.5,2.7,0.8,NaN,336.0,24.7,44.0,1010.1,NaN
2,2021-01-03,0.7,-0.3,1.6,0.0,NaN,327.0,17.1,38.9,1012.1,NaN
3,2021-01-04,0.0,-1.0,1.1,0.0,NaN,329.0,10.6,30.0,1011.4,NaN
4,2021-01-05,0.8,-0.9,2.1,0.0,NaN,338.0,6.2,30.0,1012.1,NaN
5,2021-01-06,0.8,0.2,1.4,0.0,NaN,324.0,8.0,21.0,1015.4,NaN
6,2021-01-07,0.2,-1.6,2.5,0.0,NaN,343.0,6.2,24.0,1017.1,NaN


In [4]:
print(backery.describe())

          Unnamed: 0  ticket_number       Quantity
count  234005.000000  234005.000000  234005.000000
mean   255205.035854  219201.258738       1.538377
std    147501.625992   40053.223896       1.289603
min         0.000000  150040.000000    -200.000000
25%    127979.000000  184754.000000       1.000000
50%    254573.000000  218807.000000       1.000000
75%    382911.000000  253927.000000       2.000000
max    511395.000000  288913.000000     200.000000


In [8]:
print('')
print(backery.isna().sum())

Unnamed: 0       0
date             0
time             0
ticket_number    0
Article          0
Quantity         0
UNIT_PRICE       0
dtype: int64

In [9]:
weather.isna().sum()

date      0
tavg      0
tmin      0
tmax      0
prcp      0
snow    654
wdir      2
wspd      2
wpgt     14
pres      2
tsun    669
dtype: int64

## EOD

### Utility functions

In [24]:
def extract_price(price_tag: object)->float:
  """

  Args:
    price_tag: string input for daily sale with currency sign

  Returns:
    returns price value as float
  """
  price_string=str(price_tag)
  price_value=float(price_string[:-2].replace(',','.'))
  return price_value

### Data preparation

In [63]:
weather['is_rainy']=weather['prcp']>0
hot_day_threshold=30
weather['is_hot_day'] = weather['tavg'] > hot_day_threshold
weather['is_freezing_day'] = weather['tavg'] <0

weather['is_cloudy']= weather['pres']<1010
weather['is_heavy_wind']=weather['wpgt']>30
weather

,date,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_rainy,is_hot_day,is_freezing_day,is_cloudy,is_heavy_wind
0,2021-01-01,2.7,1.5,5.7,15.2,NaN,335.0,15.5,31.0,1008.6,NaN,True,False,False,True,True
1,2021-01-02,1.3,0.5,2.7,0.8,NaN,336.0,24.7,44.0,1010.1,NaN,True,False,False,False,True
2,2021-01-03,0.7,-0.3,1.6,0.0,NaN,327.0,17.1,38.9,1012.1,NaN,False,False,False,False,True
3,2021-01-04,0.0,-1.0,1.1,0.0,NaN,329.0,10.6,30.0,1011.4,NaN,False,False,False,False,False
4,2021-01-05,0.8,-0.9,2.1,0.0,NaN,338.0,6.2,30.0,1012.1,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,2022-10-27,19.8,15.7,25.0,0.0,NaN,145.0,17.4,39.0,1023.8,NaN,False,False,False,False,True
665,2022-10-28,20.9,17.4,25.5,0.0,NaN,136.0,16.3,39.0,1024.5,NaN,False,False,False,False,True
666,2022-10-29,19.6,14.9,25.5,0.0,NaN,141.0,14.3,38.9,1022.7,NaN,False,False,False,False,True
667,2022-10-30,18.8,15.4,24.4,0.0,NaN,136.0,17.0,33.0,1020.4,NaN,False,False,False,False,True


In [27]:
backery['UNIT_PRICE_value']= backery['UNIT_PRICE'].apply(lambda x: extract_price(x))
backery["total_sale_item"]= backery['Quantity']*backery['UNIT_PRICE_value']
backery=backery.rename(columns={'Unnamed: 0':'index'})

display(backery)
print(backery.dtypes)

Unnamed: 0            int64
date                 object
time                 object
ticket_number       float64
Article              object
Quantity            float64
UNIT_PRICE           object
UNIT_PRICE_value    float64
dtype: object


In [85]:
# display(backery)
# print(backery.dtypes)
merged_data[merged_data['is_freezing_day']==True]

,date,unique_items_sold,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_rainy,is_hot_day,is_freezing_day,is_cloudy,is_heavy_wind
528,2022-07-19,56,31.2,24.4,37.5,0.0,NaN,174.0,21.2,55.0,1014.9,NaN,False,True,False,False,True
544,2022-08-04,54,31.8,24.8,38.5,0.0,NaN,180.0,14.9,44.0,1012.2,NaN,False,True,False,False,True


In [58]:

daily_count=backery.groupby(["date"])["Article"].nunique().reset_index(name='unique_items_sold')#.count()

display(daily_count)


print('Mean items sold per day:\n')
print(np.mean( daily_count['unique_items_sold'].values))

,date,unique_items_sold
0,2021-01-02,40
1,2021-01-03,38
2,2021-01-04,34
3,2021-01-05,39
4,2021-01-07,41
...,...,...
595,2022-09-26,40
596,2022-09-27,45
597,2022-09-28,46
598,2022-09-29,37


Mean items sold per day:

48.67


In [65]:
merged_data = pd.merge(daily_count, weather, on='date', how='left')
merged_data

,date,unique_items_sold,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,is_rainy,is_hot_day,is_freezing_day,is_cloudy,is_heavy_wind
0,2021-01-02,40,1.3,0.5,2.7,0.8,NaN,336.0,24.7,44.0,1010.1,NaN,True,False,False,False,True
1,2021-01-03,38,0.7,-0.3,1.6,0.0,NaN,327.0,17.1,38.9,1012.1,NaN,False,False,False,False,True
2,2021-01-04,34,0.0,-1.0,1.1,0.0,NaN,329.0,10.6,30.0,1011.4,NaN,False,False,False,False,False
3,2021-01-05,39,0.8,-0.9,2.1,0.0,NaN,338.0,6.2,30.0,1012.1,NaN,False,False,False,False,False
4,2021-01-07,41,0.2,-1.6,2.5,0.0,NaN,343.0,6.2,24.0,1017.1,NaN,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2022-09-26,40,14.8,11.8,18.3,2.8,NaN,297.0,7.8,22.2,1011.3,NaN,True,False,False,False,False
596,2022-09-27,45,13.4,10.7,16.3,3.6,NaN,240.0,10.5,33.0,1008.6,NaN,True,False,False,True,True
597,2022-09-28,46,14.2,11.2,19.2,4.3,NaN,190.0,15.9,48.0,1003.1,NaN,True,False,False,True,True
598,2022-09-29,37,12.2,9.3,15.7,24.4,NaN,321.0,6.9,18.5,1003.2,NaN,True,False,False,True,False


In [100]:
import plotly.graph_objects as go
conditions = ['is_rainy', 'is_hot_day', 'is_freezing_day', 'is_cloudy', 'is_heavy_wind']

colors = {
    # 'is_rainy': 'lightblue',
    # 'is_hot_day': 'red',
    # 'is_freezing_day': 'cyan',
    'is_cloudy': 'gray',
    'is_heavy_wind': 'lightseagreen'
}
markers = {'is_rainy': 'circle',
           'is_freezing_day':  'diamond',
           'is_hot_day':'cross'
}
marker_colors = {'is_rainy': 'darkblue',
           'is_freezing_day':  'cyan',
           'is_hot_day':'red'
}

fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x=merged_data['date'],
#     y=merged_data['unique_items_sold'],
#     mode='markers+lines',
#     name='Unique Items Sold'
# ))

for condition in markers.keys():
    if condition in merged_data.columns:
        condition_data = merged_data[merged_data[condition]]

        fig.add_trace(go.Scatter(
            x=merged_data['date'],
            y=merged_data['unique_items_sold'],
            mode='markers',
            marker=dict(
                symbol=markers[condition],
                size=10,
                color=marker_colors[condition],
                opacity=0.8,
                line=dict(width=0)
            ),
            name=condition.replace('_', ' ').title(),
            showlegend=True,
            hoverinfo='name'
        ))


shapes = []
if any(merged_data['is_cloudy']):
    shapes.append({
        'type': 'rect',
        'xref': 'x',
        # 'yref': 'paper',
        'x0': merged_data.loc[merged_data['is_cloudy'], 'date'].min(),
        'x1': merged_data.loc[merged_data['is_cloudy'], 'date'].max(),
        'y0': 0,
        'y1': 1,
        'fillcolor': colors['is_cloudy'],
        'opacity': 0.6,
        # 'layer': 'above',
        # 'line': {'width': 1},
    })


if any(merged_data['is_heavy_wind']):
    shapes.append({
        'type': 'rect',
        # 'xref': 'x',
        'yref': 'paper',
        'x0': merged_data.loc[merged_data['is_heavy_wind'], 'date'].min(),
        'x1': merged_data.loc[merged_data['is_heavy_wind'], 'date'].max(),
        'y0': merged_data['unique_items_sold'].min(),
        'y1': merged_data['unique_items_sold'].max(),
        'fillcolor': colors['is_heavy_wind'],
        'opacity':0.6,
        # 'layer': 'above',
        # 'line': {'width': 1},
    })

fig.update_layout( title='Daily Unique Items Sold', xaxis_title='Date',  yaxis_title='Unique Items Sold',  shapes=shapes )
fig.show()




In [43]:

daily_sale=backery.groupby(["date","Article"]).sum("Quantity")#.size()
daily_sale

Id  ticket_number  Quantity  \
date       Article                                                   
2021-01-02 BAGUETTE                 10519      4353814.0      46.0   
           BANETTE                   8968      3753253.0      40.0   
           BANETTINE                 1298       750529.0       6.0   
           BOULE 200G                3428       901109.0       6.0   
           BOULE 400G                4548      1351512.0      11.0   
...                                   ...            ...       ...   
2022-09-30 SPECIAL BREAD KG        511210       288863.0       1.0   
           SUCETTE                 511268       288877.0       1.0   
           TARTELETTE             1021867       577575.0       3.0   
           TRADITIONAL BAGUETTE  43441397     24550190.0     141.0   
           VIK BREAD              3577175      2021686.0      12.0   

                                 UNIT_PRICE_value  total_sale_item  
date       Article                                                  
2021-01-02 BAGUETTE                         26.10             41.4  
           BANETTE                          26.25             42.0  
           BANETTINE                         3.00              3.6  
           BOULE 200G                        6.60              6.6  
           BOULE 400G                       13.50             16.5  
...                                           ...              ...  
2022-09-30 SPECIAL BREAD KG                  5.20              5.2  
           SUCETTE                           0.30              0.3  
           TARTELETTE                        4.40              6.6  
           TRADITIONAL BAGUETTE            110.50            183.3  
           VIK BREAD                        18.90             32.4  

[29202 rows x 5 columns]